In [100]:
class BinStr:
    def __init__(self, v: str = "", lpad=0, rpad=0):
        assert all(i in {"", "0", "1"} for i in v)
        self.v = "".ljust(lpad, "0") + v + "".rjust(rpad, "0")

    def __repr__(self):
        return self.v

    def __len__(self):
        return len(self.v)

    def __iter__(self):
        yield from map(int, self.v)

    def __getitem__(self, x):
        return BinStr(self.v[x])

    def __reversed__(self):
        return BinStr(self.v[::-1])

    def __invert__(self):
        return BinStr("".join(str(1 - i) for i in self))

    def __or__(self, other):
        return BinStr(self.v + other.v)

    def __lshift__(self, other):
        assert other <= len(self)

        temp, self.v = self[:other], repr(self[other:])
        return temp

    def __int__(self):
        return sum(int(repr(self[i])) * 2 ** (len(self) - i - 1) for i in range(len(self)))

    def __add__(self, other):
        assert len(self) == len(other)

        carry = 0
        z = [None] * len(self)

        gen = (i + j + carry for i, j in zip(*map(reversed, [self, other])))

        for i, w in enumerate(gen, start=1):
            carry, z[-i] = divmod(w, 2)

        if carry:
            z.insert(0, 1)

        return BinStr("".join(map(str, z)))

    def __xor__(self, other):
        return BinStr("".join(str(int(i != j)) for i, j in zip(self, other)))

    def __mod__(self, other):
        """
        101 11010100
            101
             111 
             101
              100
              101
                110
                101
                 110
                 101
                  11
        """
        z = BinStr()

        while len(self):
            offset = len(other) - len(z)
            z |= self << offset
            z ^= other
            z.v = z.v.lstrip("0")
        
        return z

In [101]:
def test(actual, expected):
    assert repr(actual) == expected

test(~BinStr("10"), "01")
test(BinStr("11010") + BinStr("11100"), "110110")
test(BinStr("1010") ^ BinStr("0011"), "1001")

test(reversed(BinStr("0011")), "1100")

In [102]:
a = BinStr("1100010111101000")
b = BinStr("0010000000100111")

In [103]:
c = a + b

In [104]:
if len(c) > len(a):
    c = c[1:] + BinStr("1", lpad=len(c) - 2)

~c

0001100111110000

In [105]:
BinStr("11010100") % BinStr("101")

11